# Python mini-kernel

## 1. Parsing code

We probably want to use Python's own parsing to parse Python code. This is exposed by the "mini-kernel" as the `/parse` end-point. It can be used to parse code and get the result as JSON:

In [86]:
!curl -H "Content-Type:application/python" -d "42" -s http://localhost:7101/parse

{
  "_type": "Module",
  "body": [
    {
      "_type": "Expr",
      "col_offset": 0,
      "end_col_offset": 2,
      "end_lineno": 1,
      "lineno": 1,
      "value": {
        "_type": "IntConstant",
        "col_offset": 0,
        "end_col_offset": 2,
        "end_lineno": 1,
        "kind": null,
        "lineno": 1,
        "n": 42,
        "s": 42,
        "value": 42
      }
    }
  ],
  "type_ignores": []
}


In [87]:
!curl -H "Content-Type:application/python" -d "rooms=pd.read_csv(\"rooms.csv\")" -s http://localhost:7101/parse

{
  "_type": "Module",
  "body": [
    {
      "_type": "Assign",
      "col_offset": 0,
      "end_col_offset": 30,
      "end_lineno": 1,
      "lineno": 1,
      "targets": [
        {
          "_type": "Name",
          "col_offset": 0,
          "ctx": {
            "_type": "Store"
          },
          "end_col_offset": 5,
          "end_lineno": 1,
          "id": "rooms",
          "lineno": 1
        }
      ],
      "type_comment": null,
      "value": {
        "_type": "Call",
        "args": [
          {
            "_type": "StringConstant",
            "col_offset": 18,
            "end_col_offset": 29,
            "end_lineno": 1,
            "kind": null,
            "lineno": 1,
            "n": "rooms.csv",
            "s": "rooms.csv",
            "value": "rooms.csv"
          }
        ],
        "col_offset": 6,
        "end_col_offset": 30,
        "end_lineno": 1,
        "func": {
          "_type": "Attribute",
          "attr": "read_csv",
          "col

## 2. Executing code

In the notebook, the user will go over the cells and run them one by one. After they run some cells, we may be able to get more precise information about the type of the dataframe. The mini-kernel can be used to execute code. The `/exec` end-point runs some imperative code that does not return; `/eval` can be used to evaluate things that return something. Note that the mini-kernel keeps local variables between requests:

In [88]:
!curl -H "Content-Type:application/x-python-code" -d "40+2" -s http://localhost:7101/eval

42


In [89]:
!curl -H "Content-Type:application/x-python-code" -d "rooms=pd.read_csv(\"rooms.csv\")" -s http://localhost:7101/exec

In [90]:
!curl -H "Content-Type:application/x-python-code" -d "rooms.head()" -s http://localhost:7101/eval

{"room":{"0":"S3","1":"S5","2":"S9","3":"S4","4":"S8"}," floor":{"0":3,"1":2,"2":1,"3":3,"4":1}," capacity":{"0":116,"1":118,"2":118,"3":70,"4":30}}


## 3. Getting runtime types

After we run some code, we can get a list of local variables and get their runtime types. For dataframes, there is an extra function to look at the column names and types:

In [91]:
!curl -H "Content-Type:application/x-python-code" -d "a=42" -s http://localhost:7101/exec
!curl -H "Content-Type:application/x-python-code" -d "hi=\"hello\"" -s http://localhost:7101/exec

In [92]:
!curl -s http://localhost:7101/locals

{
  "a": "int",
  "hi": "str",
  "rooms": "DataFrame"
}


In [93]:
!curl -s http://localhost:7101/getcols/rooms

{"room":"str"," floor":"int64"," capacity":"int64"}
